In [1]:
%load_ext autoreload
%autoreload 2

import sentences, DCS
import json
import numpy as np

In [21]:
D = json.load(open('../NewData/data7.json'))

In [25]:
print(len(D.keys()))
d_keys = list(D.keys())
print(d_keys[0])

276
120


In [23]:
for k in d_keys:
    ki = abs(int(k))
    if(ki >= 190 and ki <= 240):
        print(k)

-200
-230
-220
-240
-190
-210


In [45]:
def _getPathValue(terminal_1, path_type, terminal_2):
    p = 0;
    return p
        
def GetPathVector(terminal_1, terminal_2):
    x = np.array([0]*len(all_path_types))
    for i in range(len(all_path_types)):
        path = all_path_types[i]
        x[i] = _getPathValue(terminal_1, path, terminal_2)        
    return x

In [46]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Counting CNG GROUPS

In [10]:
import pickle
from sentences import *
from DCS import *
import json
import numpy as np
from romtoslp import *
from collections import defaultdict

In [2]:
loaded_DCS = pickle.load(open('../Simultaneous_DCS.p', 'rb'))

In [3]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT.p', 'rb'))
len(loaded_SKT)

100078

In [58]:
def cng_Groups(sentenceObj, dDict):
    #print('SKT ANALYZE')
    #print('-'*15)
    #print(sentenceObj.sentence)
    zz = 0
    # (chunkDict, lemmaList, wordList, revMap2Chunk, qu, cngList, verbs, tuplesMain) = SentencePreprocess(sentenceObj)
    # for cid in chunkDict.keys():
    #     print('Analyzing:', rom_slp(sentenceObj.chunk[cid].chunk_name))
    #     for pos in chunkDict[cid].keys():
    #         tupIds = chunkDict[cid][pos]
    #         for ti in tupIds:
    #             print('%d :' % (pos, ), end = ' ')
    #             print(tuplesMain[ti][0][1], end=' ')
    #             for tup in tuplesMain[ti]:
    #                 print([zz, tup[2], tup[3]], end=' ')
    #                 zz += 1
    #             print('')
    #     print('-'*25)

    for chunk in sentenceObj.chunk:
        # print("Analyzing ", rom_slp(chunk.chunk_name))
        for pos in chunk.chunk_words.keys():
            for word_sense in chunk.chunk_words[pos]:
                #word_sense = fix_w_new(word_sense)
                #print(word_sense.forms)
                for form_dict in word_sense.forms:
                    if isinstance(form_dict, dict):                        
                        for key, val_arr in form_dict.items():
                            #print('Key: ', key)
                            #print('Val Array: ', val_arr)
                            for val in val_arr:
                                if isinstance(val, list):
                                    if len(val) > 1:
                                        print('Key: ', key)
                                        print('Bad Val: ', val, ' Size: ', len(val))
                                    dDict[key].add(val[0])
                                else:
                                    dDict[key].add(val)
                # for formsDict in word_sense.forms:
                #     print(getCNGs(formsDict))

In [61]:
DD = defaultdict(lambda:set())
i = 0
for key, val in loaded_SKT.items():
    i += 1
    cng_Groups(val, DD)
    if i==1000:
        break

In [63]:
pickle.dump(dict(DD), open('cng_grps_dd.p', 'wb'), protocol=2)

# # Verb CNG Groups

In [1]:
import pickle
import numpy as np
from sentences import *
from DCS import *
from collections import defaultdict
import math
from IPython.display import display
import json

### Lem2CNG and CNG2Lem Matrices

In [2]:
mat_lem2cng = pickle.load(open('../NewData/mat_lem2cng.p', 'rb'), encoding='utf-8')
mat_cng2lem = pickle.load(open('../NewData/mat_cng2lem.p', 'rb'), encoding='utf-8')

In [13]:
all_cngs = list(int(vc) for vc in list(mat_cng2lem.keys()))
print(all_cngs[0:10])
print(len(all_cngs))

[71, -35, 116, 175, -123, 28, 36, -83, -52, 99]
276


In [4]:
tense_classes = defaultdict(lambda:set())
for vc in all_cngs:
    if vc < 0:
        tense_classes[str(-math.floor(vc/10))].add((-vc)%10)
tense_classes

defaultdict(<function __main__.<lambda>>,
            {'1': {0},
             '10': {1, 2, 3, 4, 6, 7, 8, 9},
             '11': {1, 2, 3, 4, 9},
             '12': {1, 2, 3, 4, 5, 7, 9},
             '13': {1, 2, 3, 6, 9},
             '14': {1, 2, 3, 7, 9},
             '15': {0, 1, 2, 3, 4, 7, 9},
             '16': {1, 2, 3, 4, 6, 7, 8, 9},
             '17': {1, 3, 6, 9},
             '19': {0},
             '2': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
             '20': {0},
             '21': {0},
             '22': {0},
             '23': {0},
             '24': {0},
             '25': {1, 2, 3, 5, 6, 7, 9},
             '26': {0},
             '27': {1, 2, 3, 6, 8, 9},
             '28': {1, 3, 6, 9},
             '29': {3},
             '3': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
             '30': {1, 2, 3, 6, 7, 9},
             '31': {1, 2, 3, 6, 7, 8, 9},
             '4': {1, 2, 3, 4, 5, 6, 7, 8, 9},
             '5': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
             '6': {1, 2, 3, 4, 5, 6,

In [8]:
# num_person: encoding style
verb_groups = defaultdict(lambda: set())
number_forms = dict()
number_forms['sg'] = [1, 2, 3]
number_forms['du'] = [4, 5, 6]
number_forms['pl'] = [7, 8, 9]

person_forms = dict()
person_forms['fp'] = [1, 4, 7]
person_forms['sp'] = [2, 5, 8]
person_forms['tp'] = [3, 6, 9]

comb_forms = dict()
comb_forms2 = dict()
for key_num, num_vals in number_forms.items():
    for key_person, person_vals in person_forms.items():
        s = set()
        for n in num_vals:
            for p in person_vals:
                if n==p:
                    s.add(n)
        comb_forms2['%s_%s'%(key_num, key_person)] = s
comb_forms.update(number_forms)
comb_forms.update(person_forms)
comb_forms2.update(number_forms)
comb_forms2.update(person_forms)
display(comb_forms)
display(comb_forms2)

{'du': [4, 5, 6],
 'fp': [1, 4, 7],
 'pl': [7, 8, 9],
 'sg': [1, 2, 3],
 'sp': [2, 5, 8],
 'tp': [3, 6, 9]}

{'du': [4, 5, 6],
 'du_fp': {4},
 'du_sp': {5},
 'du_tp': {6},
 'fp': [1, 4, 7],
 'pl': [7, 8, 9],
 'pl_fp': {7},
 'pl_sp': {8},
 'pl_tp': {9},
 'sg': [1, 2, 3],
 'sg_fp': {1},
 'sg_sp': {2},
 'sg_tp': {3},
 'sp': [2, 5, 8],
 'tp': [3, 6, 9]}

In [11]:
verb_groups = dict()
for tense in tense_classes.keys():
    for form, mods in comb_forms.items():
        new_key = '%s_%s'%(tense, form)
        s = set()
        for modulo in mods:
            if modulo in tense_classes[tense]:
                s.add(int(tense)*10 + modulo)
        if len(s) > 0:
            verb_groups[new_key] = list(s)

for form, mods in comb_forms2.items():
    new_key = form
    s = set()
    for modulo in mods:
        for tense in tense_classes.keys():
            if modulo in tense_classes[tense]:
                s.add(int(tense)*10 + modulo)
    if len(s) > 0:
        verb_groups[new_key] = list(s)
json.dump(verb_groups, open('json_n_pickle_Files/verbgroups.json', 'w'))
# verb_groups

In [12]:
len(verb_groups)

140

# # Noun CNG Groups

In [1]:
import pickle
from sentences import *
from DCS import *
from IPython.display import display
import json

In [3]:
noungroups = json.load(open('json_n_pickle_Files/noungroups.json'))

In [5]:
len(noungroups)

54

# # PMI

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
from sentences import *
from DCS import *
from collections import defaultdict
import math
from IPython.display import display
import json
from ProbModels import *

In [3]:
# IMPORT THE DATALOADER FILES FIRST
from utilities import *
from collections import *
import pandas as pd
import numpy as np
print("Dataloader Started[Prob]...")

fullCo_oc_mat = pickleFixLoad('extras/all_dcs_lemmas_matrix_countonly.p')
unigram_counts = pickleFixLoad('../NewData/ultimate_new_lastsem/unigram_lemma_countonly.p')

cng2cngFullMat = np.mat(pickle.load(open('extras/all_dcs_cngs_matrix_countonly.p','rb'), encoding = 'latin1'))
cng2index_dict = pickleFixLoad('cng2index_dict.p')

w2w_samecng_fullmat = pickle.load(open('extras/lemmas_with_same_cngs_matrix_countonly.p', 'rb'), encoding=u'utf-8')
samecng_unigram_counts = pickle.load(open('extras/dictionary_for_lemmas_with_same_cng.p', 'rb'), encoding=u'utf-8')

v2c_fullMat = pickle.load(open('extras/verbs_vs_cngs_matrix_countonly.p', 'rb'), encoding=u'utf-8')

print("Dataloader Finished[Prob]...")

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [4]:
pbModel = ProbModels(fullCo_oc_mat = fullCo_oc_mat, unigram_counts = unigram_counts,
               cng2cngFullMat = cng2cngFullMat, cng2index_dict = cng2index_dict,
               w2w_samecng_fullmat=w2w_samecng_fullmat, samecng_unigram_counts = samecng_unigram_counts,
               v2c_fullMat = v2c_fullMat)

In [5]:
lemma_list = list(unigram_counts.keys())
print(len(lemma_list))
print(lemma_list[0], lemma_list[10], lemma_list[100])
print(pbModel.kn_word2word(lemma_list[0], lemma_list[10]))

66914
avarRaka advAra pratisrotas
2.2026413966350114e-06


In [6]:
total_co_oc_count = pbModel.total_co_oc

In [7]:
# pmi(x;y) = -log2[p(x|y)/p(x)]
def PMI(x, y, _pbModel):
    if(x in _pbModel.fullCo_oc_mat[y]):
        p_x_given_y = _pbModel.fullCo_oc_mat[y][x]/_pbModel.unigram_counts[y]
        p_x = _pbModel.unigram_counts[x]/_pbModel.unigram_total_count
        #print(p_x_given_y, ' ', p_x)
        return -math.log2(p_x_given_y/p_x)
    else:
        p_x_given_y = 1/_pbModel.unigram_counts[y]
        p_x = _pbModel.unigram_counts[x]/_pbModel.unigram_total_count
        #print(p_x_given_y, ' ', p_x)
        return -math.log2(p_x_given_y/p_x)

In [8]:
print(PMI('vEdiSa', 'SatruGAtin', pbModel))
print(PMI('SatruGAtin', 'vEdiSa', pbModel))

print(PMI('anupavyAyacCamAna', 'SatruGAtin', pbModel))
print(PMI('SatruGAtin', 'anupavyAyacCamAna', pbModel))

-19.307367596880127
-19.307367596880127
-20.629295691767492
-20.629295691767492


In [ ]:
full_PMI_Mat = defaultdict(lambda: {})
count = 0
for y, _ in fullCo_oc_mat.items():
    count += 1
    for x, _ in fullCo_oc_mat[y].items():
        # print(y, x, fullCo_oc_mat[y][x])
        # Getting PMI(x | y)
        try:
            v = PMI(x, y, pbModel)
        except:
            v = None
        if v != None:
            full_PMI_Mat[y][x] = v
    if(count % 10000 ==0):
        print("Checkpoint: ", count)
full_PMI_Mat = dict(full_PMI_Mat)
pickle.dump(full_PMI_Mat, open('../NewData/ultimate_new_lastsem/full_PMI_Mat.p', 'wb'))

In [ ]:
print(y, x)
print(pbModel.fullCo_oc_mat[y][x]/pbModel.unigram_counts[y])
print(pbModel.unigram_counts[x])

# # MST

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
from sentences import *
from DCS import *
from collections import defaultdict
import math
from IPython.display import display
import json
from word_definite import *
from romtoslp import *

In [3]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))

In [4]:
for k in list(loaded_SKT.keys())[100:]:
    loaded_SKT.pop(k, None)
    loaded_DCS.pop(k, None)

In [5]:
len(loaded_SKT)

100

### # Graph and Other Data Structures

- load skt - Done
- load dcs


- create the full graph - sktwseg_utf8
- Nodelist - Done
- Features-Adjacency Matrix
- Weight-Adjacency Matrix
- Conflict Matrix - or - dictionary (since it's sparse)


- new paths base on cng probability


### # Neural Net

In [62]:
def sigmoid(vec):
    evec = 1 + np.exp(-vec)
    return 1/evec
def relu(vec_x):
    relu_x = vec_x.copy()
    relu_x[vec_x < 0] = 0
    return relu_x
def d_relu(vec_x):
    d_relu_x = vec_x.copy()
    d_relu_x[vec_x > 0] = 1
    d_relu_x[vec_x <= 0] = 0
    return d_relu_x
class NN:
    def __init__(self, input_dimension, hidden_layer_size):
        # d: Input feature dimension i.e. the dimension of the edge feature vectors
        # n: Hidden layer size
        
        # TODO: Add Bias terms
        
        self.n = hidden_layer_size
        self.d = input_dimension
        rand_init_range = np.sqrt(6.0/(self.n+self.d))
        self.W = np.random.uniform(-rand_init_range, rand_init_range, (self.n, self.d))
        rand_init_range = np.sqrt(6.0/(self.n))
        self.U = np.random.uniform(-rand_init_range, rand_init_range, (self.n, 1))

    def Forward_Prop(self, x):
        z2 = np.matmul(self.W, x)
        a2 = relu(z2)
        s = np.matmul(self.U.transpose(), a2)
        return (z2, a2, s)
    
    def Back_Prop(self, dLdS, nodeLen, featVMat):
        N = nodeLen
        dLdU = np.zeros(self.U.shape)
        dLdW = np.zeros(self.W.shape)
        eta = 0.1
        for i in range(N):
            for j in range(N):
                if dLdS[i, j] != 0:
                    (z2, a2, s) = Forward_Prop(featVMat[i][j])
                    
                    dLdU += dLdS[i, j]*a2/N*N
                    
                    dRelu = d_relu(a2)
                    for k in range(self.n):
                        dLdW[k, :] += (dLdS[i, j]/N*N)*U[k]*dRelu[k]*featVMat[i][j]
        self.W -= eta*dLdW
        self.U -= eta*dLdU

### # Form Graph

In [42]:
mat_lem2cng_countonly = pickle.load(open('../NewData/ultimate_new_lastsem/mat_lem2cng_countonly.p', 'rb'))
mat_cng2lem_countonly = pickle.load(open('../NewData/ultimate_new_lastsem/mat_cng2lem_countonly.p', 'rb'))
mat_lem2cng_1D = pickle.load(open('../NewData/ultimate_new_lastsem/mat_lem2cng_1D.p', 'rb'))
mat_cng2lem_1D = pickle.load(open('../NewData/ultimate_new_lastsem/mat_cng2lem_1D.p', 'rb'))

In [169]:
fn = 76
sentenceObj = loaded_SKT[list(loaded_SKT.keys())[fn]]
dcsObj = loaded_DCS[list(loaded_SKT.keys())[fn]]
(chunkDict, lemmaList, wordList, revMap2Chunk, qu, cngList, verbs, tuplesMain, qc_pairs) = SentencePreprocess(sentenceObj)

nodelist = GetNodes(sentenceObj)
print("%d nodes received" % len(nodelist))
display(nodelist)

# Nodelist with only the correct_nodes
nodelist2 = GetNodes(sentenceObj)
nodelist_correct = []
search_key = 0
first_key = 0
for chunk_id in range(len(dcsObj.lemmas)):
    while nodelist2[first_key].chunk_id != chunk_id:
        first_key += 1
    for j in range(len(dcsObj.lemmas[chunk_id])):
        search_key = first_key
        
        while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != int(dcsObj.cng[chunk_id][j])):
            search_key += 1
#         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
#         print(nodelist[search_key])
        nodelist_correct.append(nodelist2[search_key])


28 nodes received


[WD_Node[C: 0, P: 0, tad @(54) => tayoH],
 WD_Node[C: 0, P: 0, tad @(55) => tayoH],
 WD_Node[C: 0, P: 0, tad @(56) => tayoH],
 WD_Node[C: 0, P: 0, tad @(154) => tayoH],
 WD_Node[C: 0, P: 0, tad @(155) => tayoH],
 WD_Node[C: 0, P: 0, tad @(156) => tayoH],
 WD_Node[C: 1, P: 0, Bnna @(3) => nirBinna],
 WD_Node[C: 1, P: 0, Bd @(-190) => nirBinna],
 WD_Node[C: 1, P: 0, nirBid @(-190) => nirBid],
 WD_Node[C: 1, P: 8, hfdaya @(29) => hfdayaH],
 WD_Node[C: 1, P: 8, hfdaya @(3) => hft],
 WD_Node[C: 1, P: 8, hfd @(29) => hft],
 WD_Node[C: 1, P: 8, hfd @(30) => hft],
 WD_Node[C: 1, P: 11, ayas @(31) => ayaH],
 WD_Node[C: 1, P: 11, ayas @(71) => ayaH],
 WD_Node[C: 1, P: 11, aya @(29) => ayaH],
 WD_Node[C: 1, P: 11, ayas @(51) => ayaH],
 WD_Node[C: 2, P: 0, tarjana @(101) => tarjanEH],
 WD_Node[C: 2, P: 0, tarj @(29) => tarjan],
 WD_Node[C: 2, P: 0, tarj @(-10) => tarjan],
 WD_Node[C: 2, P: 0, tarj @(-32) => tarja],
 WD_Node[C: 2, P: 5, na @(2) => na],
 WD_Node[C: 2, P: 6,  @(-42) => EH],
 WD_Node[

In [46]:
_edge_vector_dim = len(mat_cng2lem_1D)
_full_cnglist = list(mat_cng2lem_1D)
print(_edge_vector_dim)

276


In [63]:
neuralnet = NN(_edge_vector_dim, 100)

In [175]:
def Get_Feat_Vec_Matrix(nodelist_new, conflicts_Dict):
    nodesCount = len(nodelist_new)
    featVMat = [[None for _ in range(nodesCount)] for _ in range(nodesCount)]
    for i in range(nodesCount):
        for j in range(nodesCount):
            if j in conflicts_Dict[i] or i == j:                
                # featVMat[i][j][:] = 1e-35
                pass
            else:
                wd1 = nodelist_new[i]
                wd2 = nodelist_new[j]
                featVMat[i][j] = np.zeros((_edge_vector_dim, 1))
                # print('For ', wd1, wd2)
                for k in range(_edge_vector_dim):
                    cng_k = _full_cnglist[k]
                    # TODO: Some lemma's still missing
                    try:
                        pleft = float(mat_lem2cng_countonly[wd1.lemma][cng_k]) / mat_lem2cng_1D[wd1.lemma]
                    except KeyError:
                        pleft = 0
                    try:
                        pright = float(mat_cng2lem_countonly[cng_k][wd2.lemma]) / mat_cng2lem_1D[cng_k]
                    except KeyError:
                        pright = 0
                        
                    featVMat[i][j][k] = pleft * pright
#                         if cng_k not in mat_lem2cng_countonly[wd1.lemma].keys() or wd2.lemma not in mat_cng2lem_countonly[cng_k].keys():
#                             featVMat[i][j][k] = 0.7
#                         else:                        
#                             pleft = double(mat_lem2cng_countonly[wd1.lemma][cng_k]) / mat_lem2cng_1D[wd1.lemma]
#                             pright = double(mat_cng2lem_countonly[cng_k][wd2.lemma]) / mat_cng2lem_1D[cng_k]
#                             featVMat[i][j][k] = pleft*pright
#                             print("Here")
#                             return
                    
                    #featVMat[i][j][k] = 0.5
                
    return featVMat

# This function will be some neural network or a linear function or something of sorts
def Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist_new, conflicts_Dict):
    nodesCount = len(nodelist_new)
    WScalarMat = 1e-25 * np.ones((nodesCount, nodesCount))   
    for i in range(nodesCount):
        for j in range(nodesCount):
#             if j in conflicts_Dict[i]:
            if featVMat[i][j] is None:
                pass
            else:
                (_, _, s) = neuralnet.Forward_Prop(featVMat[i][j])
                WScalarMat[i, j]= np.maximum(1e-25, s/_edge_vector_dim)
    WScalarMat = -np.log2(WScalarMat)
    return WScalarMat

In [176]:
conflicts_Dict = Get_Conflicts(nodelist)
conflicts_Dict_correct = Get_Conflicts(nodelist_correct)

In [177]:
featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)
featVMat_correct = Get_Feat_Vec_Matrix(nodelist_correct, conflicts_Dict_correct)
#display(featVMat)

In [178]:
WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict)
WScalarMat_correct = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat_correct, nodelist_correct, conflicts_Dict_correct)

In [179]:
print(WScalarMat[0,:])
print(WScalarMat_correct[0,:])

[ 83.04820237  83.04820237  83.04820237  83.04820237  83.04820237
  83.04820237  83.04820237  83.04820237  27.98667909  24.5176578
  24.5176578   25.27979805  25.27979805  26.01418175  26.01418175
  33.79911828  26.01418175  30.97828461  32.11720179  32.11720179
  32.11720179  19.78862491  83.04820237  83.04820237  27.55670283
  22.62476649  28.236728    28.236728  ]
[ 83.04820237  27.98667909  24.5176578   30.97828461  22.62476649
  28.236728  ]


### HEAP

In [180]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [181]:
import numpy as np
from word_definite import *
import math
from heap_sp import *

In [197]:
def MST(nodelist, WScalarMat, conflicts_Dict, source):
    mst_adj_graph = np.ndarray(WScalarMat.shape, np.bool)*False
    # Reset nodes and put ids
    for id in range(len(nodelist)):
        nodelist[id].id = id
        nodelist[id].dist = np.inf
        nodelist[id].isConflicted = False
        nodelist[id].src = -1
        
    # Initialize Graph and min-Heap
    nodelist[source].dist = 0
    for neighbour in range(len(nodelist)):
        if neighbour != source:
            nodelist[neighbour].dist = WScalarMat[source][neighbour]
            nodelist[neighbour].src = source
    h = Heap(nodelist)
    
    mst_nodes = defaultdict(lambda: [])
    # Run MST only until first conflicting node is seen
    # Conflicting node will have np.inf as dist
    while True:
        nextNode = h.Pop()
        if nextNode == None:
            break
        print(nextNode.src, nextNode.id, nextNode)
        mst_nodes[nextNode.chunk_id].append(nextNode)
        if nextNode.src != -1:
            mst_adj_graph[nextNode.src, nextNode.id] = True
        nid = nextNode.id
        for conId in conflicts_Dict[nid]:
            h.Delete(nodelist[conId])
        for neighbour in range(len(nodelist)):
            if neighbour != nextNode.id:
                h.Decrease_Key(nodelist[neighbour], WScalarMat[nextNode.id][neighbour], nextNode.id)
    mst_nodes = dict(mst_nodes)
    return (mst_nodes, mst_adj_graph)

def organize_mst_nodes(mst_nodes):
    sol_lemmas = []
    sol_cngs = []
    print()
    total_tree_weight = 0
    for key, val in mst_nodes.items():
        print('{', end = '')
        for node in val:
            print(node.lemma, (node.cng, node.derived, node.src,), end = ', ')
            if node.src != -1:
                total_tree_weight += WScalarMat[node.src][node.id]
        print('};', end = ' ')        
    print()
    print('Weight of the spanning tree: ', total_tree_weight)    

In [198]:
# Get MST for the correct nodelist
source = 0
(mst_nodes_correct, mst_adj_graph_correct) = MST(nodelist_correct, WScalarMat_correct, conflicts_Dict_correct, source)
organize_mst_nodes(mst_nodes_correct)

# Get all MST
for source in range(len(nodelist)):
    print('\nSOURCE: {}'.format(source))
    (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source)    
    organize_mst_nodes(mst_nodes)
    break

-1 0 WD_Node[C: 0, P: 0, tad @(154) => tayoH]
0 4 WD_Node[C: 3, P: 0, jan @(-190) => jAta]
4 2 WD_Node[C: 1, P: 8, hfdaya @(29) => hfdayaH]
4 1 WD_Node[C: 1, P: 0, nirBid @(-190) => nirBid]
1 5 WD_Node[C: 3, P: 4, vepaTu @(29) => vepaTuH]
4 3 WD_Node[C: 2, P: 0, tarjana @(101) => tarjanEH]

{tad (154, 'tayoH', -1), }; {hfdaya (29, 'hfdayaH', 3), nirBid (-190, 'nirBid', 3), }; {tarjana (101, 'tarjanEH', 4), }; {jan (-190, 'jAta', 3), vepaTu (29, 'vepaTuH', 3), }; 
Weight of the spanning tree:  415.241011861

SOURCE: 0
-1 0 WD_Node[C: 0, P: 0, tad @(54) => tayoH]
0 21 WD_Node[C: 2, P: 5, na @(2) => na]
0 25 WD_Node[C: 3, P: 0, jan @(-190) => jAta]
25 9 WD_Node[C: 1, P: 8, hfdaya @(29) => hfdayaH]
25 8 WD_Node[C: 1, P: 0, nirBid @(-190) => nirBid]
8 26 WD_Node[C: 3, P: 4, vepaTu @(29) => vepaTuH]
9 20 WD_Node[C: 2, P: 0, tarj @(-32) => tarja]
20 22 WD_Node[C: 2, P: 6,  @(-42) => EH]

{tad (54, 'tayoH', -1), }; {hfdaya (29, 'hfdayaH', 20), nirBid (-190, 'nirBid', 26), }; {na (2, 'na', 20),

In [184]:
hh = Heap(nodelist_correct)
hh.Print()

N(0, 0.0) 

N(1, inf) N(2, inf) 

N(3, inf) N(4, inf) N(5, inf) 

In [189]:
nodelist_correct

[WD_Node[C: 0, P: 0, tad @(154) => tayoH],
 WD_Node[C: 1, P: 0, nirBid @(-190) => nirBid],
 WD_Node[C: 1, P: 8, hfdaya @(29) => hfdayaH],
 WD_Node[C: 2, P: 0, tarjana @(101) => tarjanEH],
 WD_Node[C: 3, P: 0, jan @(-190) => jAta],
 WD_Node[C: 3, P: 4, vepaTu @(29) => vepaTuH]]

In [186]:
SeeDCS(dcsObj)

DCS ANALYZE
---------------
tayor nirBinnahfdayas tarjanEr jAtavepaTuH    
[['tad'], ['nirbhid', 'hṛdaya'], ['tarjana'], ['jan', 'vepathu']]
Lemmas: ['tad', 'nirBid', 'hfdaya', 'tarjana', 'jan', 'vepaTu']
[['154'], ['-190', '29'], ['101'], ['-190', '29']]



In [152]:
# Use 37171.p
print(sentenceObj.sent_id)
SeeSentence(sentenceObj)

37171
SKT ANALYZE
---------------
tayor nirBinnahfdayas tarjanEr jAtavepaTuH    
Analyzing  tayoH
0 :  tayoH ['tad'] [{'noun': ['loc. du. n.', 'g. du. n.', 'loc. du. m.', 'g. du. m.', 'loc. du. f.', 'g. du. f.']}]
Analyzing  nirBinnahfdayas
0 :  nirBinna ['Bnna', 'Bd'] [{'compound': ['iic.']}, {'verb': ['pp.']}]
0 :  nirBid ['nirBid'] [-190]
8 :  hfdayaH ['hfdaya'] [{'noun': ['nom. sg. m.']}]
8 :  hft ['hfdaya'] [{'compound': ['iic.']}]
8 :  hft ['hfd'] [{'noun': ['nom. sg. f.', 'nom. sg. m.']}]
11 :  ayaH ['ayas'] [{'noun': ['acc. sg. n.', 'nom. sg. n.']}]
11 :  ayaH ['aya'] [{'noun': ['nom. sg. m.']}]
11 :  ayaH ['ayas'] [{'noun': ['voc. sg. n.']}]
Analyzing  tarjanEH
0 :  tarjanEH ['tarjana'] [{'noun': ['i. pl. n.']}]
0 :  tarjan ['tarj'] [{'verbform': ['"http:sanskrit.inria.frMW106.html#H_tarj"'], 'verb': [['ppr. [1] ac.']], 'noun': ['nom. sg. m.']}]
0 :  tarja ['tarj'] [{'verb': ['imp. [1] ac. sg. 2']}]
5 :  na ['na'] [{'indeclinable': ['part.']}]
6 :  EH [''] [{'verb': ['impft. [